In [11]:
### Pour voir les métiers recommandés à un élève, on utilise la fonction show_score(eleve, n_metiers)
### --> eleve : liste des critères choisis par l'élève
### --> n_metiers : nombre de métiers à montrer

import io
import os
import json
import csv
import pandas as pd

In [12]:
### Préparation des chemins utiles
# Main directory
dir_path = '/home/timothee/PycharmProjects/ImpalaPoleEmploi2/Impala/POC'
# Dossier des données
data_dir_path = os.path.join(dir_path, 'Donnees')
# Caractéristiques de chaque cluster :
main_feature_cluster = os.path.join(data_dir_path, 'main_features_clusters.csv')
# Métiers de chaque cluster :
jobs_clusters = os.path.join(data_dir_path, 'jobs_clusters.csv')
# Tableau des métiers-variables:
df_jobs_file = os.path.join(data_dir_path, 'PE_niv23.csv')
# Tableau des métiers-variables normalisé:
df_clusters_file = os.path.join(data_dir_path, 'df_clusters.csv')


### Préparation des données des clusters
feature_cluster = {}
raw_csv = pd.read_csv(main_feature_cluster)
i = 0
for row in raw_csv.itertuples():
    try:
        if int(row[1]) in range(101):
            i += 1
        feature_cluster[i] = []
    except:
        feature = row[1]
        feature_cluster[i].append(feature)

In [15]:
# Elèves fictifs avec les affinités qu'on pourrait imaginer(technicien, manager, artiste)
# On peut rajouter l'élève obtenu avec le POC "Choix des critères par l'utilisateur"
yara = ['Communiquer',
 'Proposer, vendre un produit, un service',
 'Artistique',
 'Compétences managériales',
 'Marketing / Strategie']

hector = ['Faire un schéma, un plan, un dessin',
 'Enquêter, rechercher',
 'Encadrer, gérer une équipe',
 'Gérer une entreprise, un service',
 'Prendre des décisions, définir une stratégie',
 'Coordonner un projet, des actions, une formation',
 'Mentales']

jerome = ['Concevoir une offre, un projet, un produit, un processus',
 'Faire un schéma, un plan, un dessin',
 'Evaluer, étudier, analyser',
 'Enquêter, rechercher',
 'Cuisiner',
 'Produire, participer à des travaux manuels',
 "Prendre soin d'un animal",
 'Prendre soin de personnes',
 'Manipuler un outil, un appareil, un instrument',
 'Ranger, conditionner, trier',
 'Artistique',
 'Littérature, philosophie',
 'Linguistique',
 'Résolution de problème',
 'Gestion de conflit',
 'Matières naturelles',
 'Machines/outils/appareils',
 'Instruments de communication/multimédia',
 'Techniques',
 'Mentales']

eleve = ['Evaluer, étudier, analyser',
         'Enquêter, rechercher',
         'Littérature, philosophie',
         'Artistique',
         'Linguistique',
         "Techniques d'entretien, Communication/Ecrit/oral",
         'Résolution de problème',
         "Communication / Traitement de l'information",
         'Transport/Logistique']
manager = ["Conseiller, donner un avis d'expert",
            "Identifier un besoin du client",
            "Gérer un budget, exécuter une tâche de gestion comptable, financière",
           "Organiser un évènement, superviser une opération",
           "Coordonner un projet, des actions, une formation",
           "Encadrer, gérer une équipe",
           "Prendre des décisions, définir une stratégie",
           "Synthétiser, traiter une information",
           "Compétences managériales",
           "Résolution de problème",
           "Techniques d'entretien, Communication/Ecrit/oral",
           "Communication / Traitement de l'information",
           "Gestion, finance, comptabilité",
           "Mathématiques/Statistiques"]
technicien = ["Réaliser une activité de gros oeuvre (construire un batiment, une installation)",
                 "Réaliser une activité du second oeuvre (aménager, peindre, poser un revêtement)",
                 "Gérer une entreprise, un service",
                 "Conduire un véhicule, un engin",
                 "Conduire une machine",
                 "Manipuler un outil, un appareil, un instrument",
                 "Porter une charge par ses propres moyens",
                 "Effectuer des mesures",
                 "Diagnostiquer une panne",
                 "Réparer une machine, du matériel",
                 "Avec outil",
                 "Physiques",
                 "Techniques",
                 "Techniques, procédés de manutention",
                 "Gros oeuvre / Extraction",
                 "Machines/outils/appareils",
                 "Materiel et équipement",
                 "Engins/véhicules"]
artiste = ["Décorer, agencer",
          "Concevoir un projet artistique",
          "Jouer de la musique, chanter",
          "Jouer un rôle, défiler, danser",
          "Pratiquer un art plastique",
          "Faire un schéma, un plan, un dessin",
          "Communiquer",
          "Traiter des images, des sons, des lumières, des contenus multimédias",
          "Multimédia",
          "Artistique",
          "Littérature, philosophie"]


In [7]:
# Transforme les variables de la même manière que sur R
def cast_eleve_R(eleve_to_transform):
    eleve = []
    for feature in eleve_to_transform:
        feature = feature.replace(', ','..')
        feature = feature.replace('(', '.')
        feature = feature.replace(')', '.')
        feature = feature.replace("'", '.')
        feature = feature.replace("/", '.')
        feature = feature.replace(' ', '.')
        eleve.append(feature)
    return(eleve)

# Scoring de chaque groupe de métiers en fonction du nombre de critères en commun avec ceux de l'élève
def score_cluster(eleve_R):
    score_clusters = {}
    for k in range(100):
        score = 0
        for feature in eleve_R:  
            if feature in feature_cluster[k+1]:
                score += 1
        score_clusters[k+1]= score
    return(score_clusters)

#score_man = score_cluster(manager_R)

# Préparation des métiers des clusters
def score_jobs(score):
    jobs_clust = pd.read_csv(jobs_clusters)
    final_clusters = pd.DataFrame()
    for k,v in score.items():
        subset = jobs_clust[jobs_clust.clusters==k]
        subset = subset.loc[:,["intitule","clusters"]]
        subset["score"]= [v]*len(subset)
        final_clusters = final_clusters.append(subset)
    return(final_clusters.sort_values(by="score",axis=0,ascending=False))

def show_score(eleve, n_metiers=20):
    eleve_R = cast_eleve_R(eleve)
    score_clust = score_cluster(eleve_R)
    score = score_jobs(score_clust)
    print("--> Préférences de l'élève : \n")
    for item in eleve:
        print(item)
    print("\n --> Métiers recommandés :")
    print(score[0:n_metiers])

In [16]:
show_score(yara, 20)

--> Préférences de l'élève : 

Communiquer
Proposer, vendre un produit, un service
Artistique
Compétences managériales
Marketing / Strategie

 --> Métiers recommandés :
                                              intitule  clusters  score
47                                            Stylisme        16      2
102                      Relation technico-commerciale        34      2
209                                  Design industriel        16      2
137      Conception - aménagement d'espaces intérieurs        16      2
483                              Stratégie commerciale        88      2
93             Vente en gros de matériel et équipement        29      2
460          Direction de petite ou moyenne entreprise        88      2
459  Direction de grande entreprise ou d'établissem...        88      2
376  Management de structure de santé, sociale ou p...        88      2
80                     Vente en gros de produits frais        29      2
99          Relation commerciale en ven

In [14]:
show_score(hector, 20)

--> Préférences de l'élève : 

Faire un schéma, un plan, un dessin
Enquêter, rechercher
Encadrer, gérer une équipe
Gérer une entreprise, un service
Prendre des décisions, définir une stratégie
Coordonner un projet, des actions, une formation
Mentales

 --> Métiers recommandés :
                                              intitule  clusters  score
23                                Equipage de la pêche         5      3
522  Direction d'exploitation des transports routie...        98      3
387     Direction de la sécurité civile et des secours         5      3
390                         Sécurité civile et secours         5      3
502                       Direction de site logistique        98      3
179                   Éducation en activités sportives         5      3
514  Exploitation des opérations portuaires et du t...        98      3
515                  Exploitation du transport fluvial        98      3
510   Direction d'escale et exploitation aéroportuaire        98      3
5

In [10]:
show_score(jerome, 20)

--> Préférences de l'élève : 

Concevoir une offre, un projet, un produit, un processus
Faire un schéma, un plan, un dessin
Evaluer, étudier, analyser
Enquêter, rechercher
Cuisiner
Produire, participer à des travaux manuels
Prendre soin d'un animal
Prendre soin de personnes
Manipuler un outil, un appareil, un instrument
Ranger, conditionner, trier
Artistique
Littérature, philosophie
Linguistique
Résolution de problème
Gestion de conflit
Matières naturelles
Machines/outils/appareils
Instruments de communication/multimédia
Techniques
Mentales

 --> Métiers recommandés :
                                              intitule  clusters  score
166                    Taille et décoration de pierres        10      4
238          Réalisation d'ouvrages décoratifs en bois        10      4
237                     Réalisation de meubles en bois        10      4
30                         Création en arts plastiques        10      4
243  Conduite de machine de fabrication de produits...        45 

In [11]:
show_score(eleve,20)

--> Préférences de l'élève : 

Evaluer, étudier, analyser
Enquêter, rechercher
Littérature, philosophie
Artistique
Linguistique
Techniques d'entretien, Communication/Ecrit/oral
Résolution de problème
Communication / Traitement de l'information
Transport/Logistique

 --> Métiers recommandés :
                                              intitule  clusters  score
115                                      Communication        38      3
117                             Coordination d'édition        38      3
384    Gestion de l'information et de la documentation        38      3
431          Présentation de spectacles ou d'émissions        38      3
114                     Ecriture d'ouvrages, de livres        38      3
501  Conception et organisation de la chaîne logist...        67      2
257             Management et ingénierie de production        67      2
221  Intervention technique en gestion industrielle...        67      2
130     Préparation et correction en édition et presse     

In [22]:
### Pour voir les recommandations d'un élève, on utilise la fonction show_score(eleve, n_metiers)
### --> eleve : liste des critères choisis par l'élève
### --> n_metiers : nombre de métiers à montrer
show_score(manager,20)

--> Préférences de l'élève : 

Conseiller, donner un avis d'expert
Identifier un besoin du client
Gérer un budget, exécuter une tâche de gestion comptable, financière
Organiser un évènement, superviser une opération
Coordonner un projet, des actions, une formation
Encadrer, gérer une équipe
Prendre des décisions, définir une stratégie
Synthétiser, traiter une information
Compétences managériales
Résolution de problème
Techniques d'entretien, Communication/Ecrit/oral
Communication / Traitement de l'information
Gestion, finance, comptabilité
Mathématiques/Statistiques

 --> Métiers recommandés :
                                              intitule  clusters  score
410            Orientation scolaire et professionnelle        90      3
182                Conception de produits touristiques        36      3
120                         Traduction, interprétariat        40      3
130     Préparation et correction en édition et presse        40      3
119                        Organisation

In [23]:
show_score(technicien, 20)

--> Préférences de l'élève : 

Réaliser une activité de gros oeuvre (construire un batiment, une installation)
Réaliser une activité du second oeuvre (aménager, peindre, poser un revêtement)
Gérer une entreprise, un service
Conduire un véhicule, un engin
Conduire une machine
Manipuler un outil, un appareil, un instrument
Porter une charge par ses propres moyens
Effectuer des mesures
Diagnostiquer une panne
Réparer une machine, du matériel
Avec outil
Physiques
Techniques
Techniques, procédés de manutention
Gros oeuvre / Extraction
Machines/outils/appareils
Materiel et équipement
Engins/véhicules

 --> Métiers recommandés :
                                              intitule  clusters  score
446                               Machinerie spectacle        94      6
496             Magasinage et préparation de commandes        95      6
495                                       Déménagement        94      6
517  Conduite de transport de marchandises sur long...        94      6
521  Condu

In [24]:
show_score(artiste, 20)

--> Préférences de l'élève : 

Décorer, agencer
Concevoir un projet artistique
Jouer de la musique, chanter
Jouer un rôle, défiler, danser
Pratiquer un art plastique
Faire un schéma, un plan, un dessin
Communiquer
Traiter des images, des sons, des lumières, des contenus multimédias
Multimédia
Artistique
Littérature, philosophie

 --> Métiers recommandés :
                                            intitule  clusters  score
128            Encadrement des industries graphiques        44      5
117                           Coordination d'édition        38      5
131                                        Prépresse        44      5
132                                     Reprographie        44      5
133    Intervention technique en industrie graphique        44      5
439   Réalisation cinématographique et audiovisuelle        93      5
436              Mise en scène de spectacles vivants        93      5
431        Présentation de spectacles ou d'émissions        38      5
384  Gestion